# Final Colab Notebook: Multi-Agent Vulnerability Detection & Response

**Overview:** This notebook demonstrates a lightweight multi-agent CAI pipeline using Bandit + Semgrep (Scanner), a Hugging Face model for Analyst & Responder, and a Streamlit UI exposed via ngrok.

**Important:** Use synthetic code only. Do NOT paste production secrets. You'll be prompted securely for your Hugging Face token and ngrok authtoken.


In [ ]:
# Install required packages
!pip install --quiet streamlit pyngrok huggingface_hub bandit semgrep
print('Installed dependencies')

In [ ]:
from huggingface_hub import InferenceClient
import getpass, os

HF_TOKEN = getpass.getpass('Enter your Hugging Face API token (hf_...): ' )
os.environ['HF_TOKEN'] = HF_TOKEN  # stored for main.py to read
client = InferenceClient(token=HF_TOKEN)
MODEL = 'HuggingFaceH4/zephyr-7b-beta'
print('Hugging Face client ready. Model set to', MODEL)

In [ ]:
%%bash
mkdir -p sample_app
cat > sample_app/vulnerable.py <<'PY'
import os
import sqlite3

# Hardcoded secret (Bandit should flag this)
API_KEY = '12345-SECRET-KEY'

def run_command(user_input):
    # Command injection vulnerability
    os.system('echo ' + user_input)

def insecure_db_lookup(user):
    # SQL injection vulnerability
    conn = sqlite3.connect('users.db')
    cur = conn.cursor()
    query = "SELECT * FROM users WHERE username = '%s'" % user
    cur.execute(query)
    return cur.fetchall()

def main():
    u = input('name: ' )
    run_command(u)
    insecure_db_lookup(u)

if __name__ == '__main__':
    main()
PY
echo 'Created sample_app/vulnerable.py' && ls -l sample_app

In [ ]:
%%writefile main.py
import subprocess, json, os
from huggingface_hub import InferenceClient

# Read token from environment (set earlier in notebook)
HF_TOKEN = os.environ.get('HF_TOKEN')
if not HF_TOKEN:
    raise RuntimeError('HF_TOKEN not set in environment. Run the token cell and try again.')

client = InferenceClient(token=HF_TOKEN)
MODEL = 'HuggingFaceH4/zephyr-7b-beta'

def scanner_agent():
    subprocess.run('bandit -r sample_app -f json -o bandit.json', shell=True, check=False)
    subprocess.run('semgrep --config=p/ci sample_app --json --output semgrep.json', shell=True, check=False)
    try:
        bandit_results = json.load(open('bandit.json'))
    except Exception:
        bandit_results = {'results': []}
    try:
        semgrep_results = json.load(open('semgrep.json'))
    except Exception:
        semgrep_results = {'results': []}
    return bandit_results, semgrep_results

def analyst_agent(bandit, semgrep):
    findings = {'bandit': bandit.get('results', []), 'semgrep': semgrep.get('results', [])}
    prompt = (
        'You are a security analyst. Prioritize and summarize these vulnerability findings for a developer. '
        'Use High/Medium/Low priorities and include a one-line rationale and a short remediation suggestion for each finding. Do not provide exploit steps.\n\n'
        + json.dumps(findings, indent=2)
    )
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{'role': 'user', 'content': prompt}],
        max_tokens=400
    )
    try:
        return response.choices[0].message['content']
    except Exception:
        return str(response)

def responder_agent(analysis):
    prompt = (
        'You are a security responder. Based on the analysis below, produce a concise remediation plan with (1) steps to fix, (2) a short developer message to include in a PR/ticket, and (3) suggested ticket metadata (severity, labels).\n\nAnalysis:\n' + str(analysis)
    )
    response = client.chat.completions.create(
        model=MODEL,
        messages=[{'role': 'user', 'content': prompt}],
        max_tokens=400
    )
    try:
        return response.choices[0].message['content']
    except Exception:
        return str(response)

def coordinator():
    bandit, semgrep = scanner_agent()
    analysis = analyst_agent(bandit, semgrep)
    response = responder_agent(analysis)
    return bandit, semgrep, analysis, response

In [ ]:
%%writefile app.py
import streamlit as st
from main import coordinator

st.set_page_config(page_title='AI-Powered Vulnerability Detection', layout='wide')
st.title('🤖 Multi-Agent Vulnerability Detection & Response')

st.markdown('''
This demo runs Bandit + Semgrep → Analyst Agent (LLM) → Responder Agent (LLM).

**How to use**:
- Click **Run Multi-Agent Scan** to start scanning and generate analysis & remediation.
''')

if st.button('🚀 Run Multi-Agent Scan'):
    with st.spinner('🔍 Running scanner agents (Bandit & Semgrep)...'):
        bandit, semgrep, analysis, response = coordinator()

    st.subheader('🔍 Scanner Agent Findings')
    st.json({'bandit': bandit.get('results', []), 'semgrep': semgrep.get('results', [])})

    st.subheader('🧠 Analyst Agent Summary')
    st.write(analysis)

    st.subheader('🚑 Responder Agent Plan')
    st.write(response)

In [ ]:
# Start Streamlit and ngrok
from pyngrok import ngrok
import getpass, subprocess, time, os

NGROK_TOKEN = getpass.getpass('Enter your ngrok authtoken (optional, press Enter to skip): ' )
if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)

# Run Streamlit in background
subprocess.Popen(['streamlit', 'run', 'app.py', '--server.port', '10000'])
time.sleep(2)
url = ngrok.connect(10000)
print('✅ Your Streamlit app is live at:', url.public_url)

### Troubleshooting

- If you see `401 Unauthorized` from Hugging Face: re-generate your HF token and re-run the HF token cell.

- If ngrok fails: ensure you set an ngrok token or try re-running the ngrok cell.

- If `main` import fails: ensure you executed the cell that wrote `main.py` before running app cells.

### Quick run checklist

1. Run the install cell.
2. Run the HF token cell and paste token securely.
3. Run the sample_app creation cell.
4. Run the `main.py` and `app.py` write cells.
5. Run the ngrok launch cell and open the printed public URL.
